In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import openrouteservice
from shapely.geometry import Point, Polygon, MultiPolygon

In [2]:
# Load the ORS client with your API key
client = openrouteservice.Client(key='5b3ce3597851110001cf62488c71fa847a704dee913c2695723d7af2')

In [3]:
# Load the rental properties data (limit to 20 rows for testing)
properties_df = pd.read_csv('../data/landing/scraped_properties.csv').head(10)

# Convert property coordinates into GeoDataFrame
properties_df['coordinates'] = properties_df['coordinates'].apply(lambda x: tuple(map(float, x.split(','))))
properties_gdf = gpd.GeoDataFrame(properties_df, geometry=properties_df['coordinates'].apply(lambda x: Point(x[1], x[0])))

In [4]:
# Load public transport shapefiles for train and tram stops
transport_stops_gdf = gpd.read_file('../data/map/Public Transport Victoria/gis_osm_transport_a_free_1.shp')

# Filter only relevant public transport stops (train stations and tram stops)
train_stops_gdf = transport_stops_gdf[transport_stops_gdf['fclass'].isin(['railway_station', 'railway_halt'])]
tram_stops_gdf = transport_stops_gdf[transport_stops_gdf['fclass'] == 'tram_stop']

# Function to convert Polygon or MultiPolygon to its centroid
def get_centroid(geom):
    if isinstance(geom, (Polygon, MultiPolygon)):
        return geom.centroid
    return geom

# Adjust to use centroids for any Polygon or MultiPolygon
train_stops_gdf['geometry'] = train_stops_gdf['geometry'].apply(get_centroid)
tram_stops_gdf['geometry'] = tram_stops_gdf['geometry'].apply(get_centroid)

/Users/salehakhalid/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/salehakhalid/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [5]:
# Function to get the nearest stops within a buffer or nearest N stops
def get_nearest_stops(property_point, transport_stops, num_stops=1, max_distance=10000):
    # Create a GeoSeries with the property point and a buffer around it (in meters)
    property_buffer = property_point.buffer(max_distance)  # Adjust buffer distance as needed

    # Filter the transport stops to those within the buffer
    nearby_stops = transport_stops[transport_stops.geometry.within(property_buffer)]

    # If there are more than num_stops, filter to nearest
    if len(nearby_stops) > num_stops:
        nearby_stops['distance'] = nearby_stops.geometry.distance(property_point)
        nearby_stops = nearby_stops.nsmallest(num_stops, 'distance')

    return nearby_stops

In [6]:
# Function to calculate the driving distance from a property to the nearest transport stop
def calculate_nearest_transport(property_point, transport_points):
    # Get the nearest stops (limit to 1 for the nearest stop)
    nearest_stops = get_nearest_stops(property_point, transport_points, num_stops=1)

    # If no nearby stops are found, return None
    if nearest_stops.empty:
        return None
    
    # Get the coordinates for the property and the nearest stop
    coords_property = (property_point.x, property_point.y)
    coords_nearest_stop = (nearest_stops.geometry.iloc[0].x, nearest_stops.geometry.iloc[0].y)

    # Calculate the distance matrix using the ORS API (driving-car profile)
    try:
        distances = client.distance_matrix(
            locations=[coords_property, coords_nearest_stop],
            profile='driving-car',
            metrics=['distance'],
            units='m'
        )

        # Return the distance between the property and the nearest stop
        return distances['distances'][0][1]  # Distance from property to nearest stop

    except openrouteservice.exceptions.ApiError as e:
        print(f"ORS API Error: {e}")
        return None

In [7]:
# For each property, calculate the distance to the nearest train stop and tram stop
properties_gdf['nearest_train_distance'] = properties_gdf['geometry'].apply(lambda x: calculate_nearest_transport(x, train_stops_gdf))
properties_gdf['nearest_tram_distance'] = properties_gdf['geometry'].apply(lambda x: calculate_nearest_transport(x, tram_stops_gdf))

# Save the updated property data with distances to a CSV (just for 20 rows)
properties_gdf.to_csv('../data/raw/properties_with_distances_test.csv', index=False)

/var/folders/bz/yjywszk15kxc8d5vv73k9xcc0000gn/T/ipykernel_89848/1188733481.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearby_stops['distance'] = nearby_stops.geometry.distance(property_point)
/var/folders/bz/yjywszk15kxc8d5vv73k9xcc0000gn/T/ipykernel_89848/1188733481.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearby_stops['distance'] = nearby_stops.geometry.distance(property_point)
/var/folders/bz/yjywszk15kxc8d5vv73k9xcc0000gn/T/ipykernel_89848/1188733481.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearby_stops['distance'] = nearby_stops.ge